# PRS CS 

## 1. GWAS sumstats prep 

### 1.1 Read in data 

###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Make sure to be in R kernerl 
##### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (for more information on how to format files - https://github.com/getian107/PRScs)

####  T2D Phenotype - EUR and AFR (separate)

## *EUR Ancestry*

In [ ]:
library(data.table)
# unchanged summ stats - reading in original files off server 
diabetesEURData <- fread('/project/rkemberlab/summary_stats/EUR/Suzuki_2024_T2D_EUR-v2.txt')
head(diabetesEURData)

### 1.1.1 Merge with rs SNP data if needed

##### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Check with summ stats methods to see what genome ref used - in this case they used hg19 which is the same as the following snp ref being used 
##### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Merge twice (effectAllele - A1 , NonEffectAllele - A2) (effectAllele - A2, NonEffectAllele - A1) 

In [ ]:
snpinfoEURhg19 <- fread('/project/rkemberlab/tools/PRScs/PRScsScore/ldblk_1kg_eur/snpinfo_1kg_hm3')
head(snpinfoEUR)
diabEUR_mergedRS_1 <- merge(diabetesEURData, snpinfoEURhg19, by.x = c('Chromsome', 'Position', 'EffectAllele', 'NonEffectAllele'), by.y = c('CHR', 'BP', 'A1', 'A2'))
diabEUR_mergedRS_2 <- merge(diabetesEURData, snpinfoEURhg19, by.x = c('Chromsome', 'Position', 'EffectAllele', 'NonEffectAllele'), by.y = c('CHR', 'BP', 'A2', 'A1'))
nrow(diabEUR_mergedRS_1)
nrow(diabEUR_mergedRS_2)
diabEUR_mergedRS <- rbind(diabEUR_mergedRS_1, diabEUR_mergedRS_2)
head(diabEUR_mergedRS)

### 1.2 Determine neff or sample size for --N param

In [ ]:
N <- diabEUR_mergedRS$Neff
N <- as.numeric(N)
N <- na.omit(N)
max(N)

### 1.3 Extract and Rename Columns and  Upper the allele

In [ ]:
diabEURdf <- diabEUR_mergedRS[,c('SNP','EffectAllele','NonEffectAllele','Beta','Pval')] #ouput file names
names(diabEURdf) <- c('SNP','A1','A2','BETA','P')
# Upper the Allele
diabEURdf$A1 <- toupper(diabEURdf$A1)
diabEURdf$A2 <- toupper(diabEURdf$A2)
fwrite(diabEURdf,'/project/rkemberlab/summary_stats/reformatted_sumstats/prscs_prscsx/T2D_eur_Suzuki2024_reformatted_sumstats.txt',row.names = F,col.names = T,sep = '\t')
head(diabEURdf)

## 2. run PRScs code  

##### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Change into bash kernel 

### 2.1.1 Check run_prscs_eur script 

In [ ]:
cat /project/rkemberlab/scripts/run_prscs_eur.sh

### 2.1.2 Check prscs_eur script 

In [ ]:
cat /project/rkemberlab/scripts/prscs_eur.sh 

### 2.2 Run bash script 
##### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Command line prompt and param order :
##### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; [script] [path to reformatted sum stats] [sample size] [path to out dir] [file prefix] 

In [ ]:
sh /project/rkemberlab/scripts/run_prscs_eur.sh /project/rkemberlab/summary_stats/reformatted_sumstats/prscs_prscsx/T2D_eur_Suzuki2024_reformatted_sumstats.txt 751755 /home/mlreed/PRSCalcs/EUR/T2D/ Suzuki2024T2D_EUR   

### 2.3 Merge the PRScs outputs for each chrom into one file 

In [ ]:
cat /home/mlreed/PRSCalcs/EUR/T2D/prscs_EUR_Suzuki2024T2D_EUR_pst_eff_a1_b0.5_phiauto_chr* > /home/mlreed/PRSCalcs/EUR/T2D/prscs_EUR_Suzuki2024T2D_pst_eff_a1_b0.5_phiauto_chrALL.txt

### 2.4 Calculate PRS with plink

In [ ]:
module load plink-1.90b3b
plink \
--bfile /project/rkemberlab/datasets/PMBB/Genotype/PMBB-Release-2020-2.0/PMBB-Release-2020-2.0_genetic_imputed-topmed-r2_chrAll_1kg_hm3_eur_newID_EUR \
--score /home/mlreed/PRSCalcs/EUR/T2D/prscs_EUR_Suzuki2024T2D_pst_eff_a1_b0.5_phiauto_chrALL.txt 2 4 6 \
--out PRS_prscs_EUR_T2D

## *AFR Ancestry*

##### Change back to R 

In [ ]:
library(data.table)
diabetesAFRData <- fread('/project/rkemberlab/summary_stats/AFR/Suzuki_2024_T2D_AFR-v2.txt')
snpinfoAFRhg19 <- fread('/project/rkemberlab/tools/PRScs/PRScsScore/ldblk_1kg_afr/snpinfo_1kg_hm3')
head(snpinfoAFRhg19)
diabAFR_mergedRS_1 <- merge(diabetesAFRData, snpinfoAFRhg19, by.x = c('Chromsome', 'Position', 'EffectAllele', 'NonEffectAllele'), by.y = c('CHR', 'BP', 'A1', 'A2'))
diabAFR_mergedRS_2 <- merge(diabetesAFRData, snpinfoAFRhg19, by.x = c('Chromsome', 'Position', 'EffectAllele', 'NonEffectAllele'), by.y = c('CHR', 'BP', 'A2', 'A1'))
nrow(diabAFR_mergedRS_1)
nrow(diabAFR_mergedRS_2)
diabAFR_mergedRS <- rbind(diabAFR_mergedRS_1, diabAFR_mergedRS_2)
nrow(diabAFR_mergedRS)
head(diabAFR_mergedRS)
N <- diabAFR_mergedRS$Neff
N <- as.numeric(N)
N <- na.omit(N)
max(N)

In [ ]:
diabAFRdf <- diabAFR_mergedRS[,c('SNP','EffectAllele','NonEffectAllele','Beta','Pval')] #ouput file names
names(diabAFRdf) <- c('SNP','A1','A2','BETA','P')
 # Upper the Allele
diabAFRdf$A1 <- toupper(diabAFRdf$A1)
diabAFRdf$A2 <- toupper(diabAFRdf$A2)
fwrite(diabAFRdf,'/project/rkemberlab/summary_stats/reformatted_sumstats/prscs_prscsx/T2D_afr_Suzuki2024_reformatted_sumstats.txt',row.names = F,col.names = T,sep = '\t')
head(diabAFRdf)

##### Change back to bash

In [ ]:
cat /project/rkemberlab/scripts/prscs_afr.sh 

In [ ]:
cat /project/rkemberlab/scripts/run_prscs_afr.sh 

In [ ]:
sh /project/rkemberlab/scripts/run_prscs_afr.sh /project/rkemberlab/summary_stats/reformatted_sumstats/prscs_prscsx/T2D_afr_Suzuki2024_reformatted_sumstats.txt 130709 /home/mlreed/PRSCalcs/AFR/T2D/ Suzuki2024T2D_AFR   

In [ ]:
cat /home/mlreed/PRSCalcs/AFR/T2D/prscs_AFR_Suzuki2024T2D_AFR_pst_eff_a1_b0.5_phiauto_chr* > /home/mlreed/PRSCalcs/AFR/T2D/prscs_AFR_Suzuki2024T2D_pst_eff_a1_b0.5_phiauto_chrALL.txt

In [ ]:
module load plink-1.90b3b
plink \
--bfile /project/rkemberlab/datasets/PMBB/Genotype/PMBB-Release-2020-2.0/PMBB-Release-2020-2.0_genetic_imputed-topmed-r2_chrAll_1kg_hm3_afr_newID_AFR \
--score /home/mlreed/PRSCalcs/AFR/T2D/prscs_AFR_Suzuki2024T2D_pst_eff_a1_b0.5_phiauto_chrALL.txt 2 4 6 \
--out PRS_prscs_AFR_T2D

## Cardio Phenotype - EUR Only 

#####  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Switch to R kernel 

In [ ]:
library(data.table)
cardioData <- fread('/project/rkemberlab/summary_stats/EUR/Nelson_2017_Cardio_EUR.txt')

In [ ]:
head(cardioData)
snpinfoEUR <- fread('/project/rkemberlab/tools/PRScs/PRScsScore/ldblk_1kg_eur/snpinfo_1kg_hm3')
head(snpinfoEUR)

In [ ]:
cardio_mergedRS_1 <- merge(cardioData, snpinfoEUR, by.x = c('chr', 'bp_hg19', 'effect_allele', 'noneffect_allele'), by.y = c('CHR', 'BP', 'A1', 'A2'))
cardio_mergedRS_2 <- merge(cardioData, snpinfoEUR, by.x = c('chr', 'bp_hg19', 'effect_allele', 'noneffect_allele'), by.y = c('CHR', 'BP', 'A2', 'A1'))

In [ ]:
nrow(cardio_mergedRS_1)
nrow(cardio_mergedRS_2)
cardio_mergedRS <- rbind(cardio_mergedRS_1, cardio_mergedRS_2)
nrow(cardio_mergedRS)
head(cardio_mergedRS)

In [ ]:
cardioNeff <- (4*(60801*123504))/(60801+123504)
cardioNeff

In [ ]:
cardioDf <- cardio_mergedRS[,c('SNP','effect_allele','noneffect_allele','beta','het_pvalue')] #ouput file names
names(cardioDf) <- c('SNP','A1','A2','BETA','P')
# Upper the Allele
cardioDf$A1 <- toupper(cardioDf$A1)
cardioDf$A2 <- toupper(cardioDf$A2)
fwrite(cardioDf,'/project/rkemberlab/summary_stats/reformatted_sumstats/prscs_prscsx/cardio_eur_Nelson2017_reformatted_sumstats.txt',row.names = F,col.names = T,sep = '\t')
head(cardioDf)

#####  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Switch to bash kernel 

In [ ]:
cat /project/rkemberlab/scripts/run_prscs_eur.sh

In [ ]:
sh /project/rkemberlab/scripts/run_prscs_eur.sh /project/rkemberlab/summary_stats/reformatted_sumstats/prscs_prscsx/cardio_eur_Nelson2017_reformatted_sumstats.txt 162973 /home/mlreed/PRSCalcs/EUR/Cardio/ Nelson2017EUR_Cardio  

In [ ]:
cat /home/mlreed/PRSCalcs/EUR/Cardio/prscs_EUR_Nelson2017EUR_Cardio_pst_eff_a1_b0.5_phiauto_chr* > /home/mlreed/PRSCalcs/EUR/Cardio/prscs_EUR_Nelson2017EUR_Cardio_pst_eff_a1_b0.5_phiauto_chrALL.txt

In [ ]:
module load plink-1.90b3b
plink \
--bfile /project/rkemberlab/datasets/PMBB/Genotype/PMBB-Release-2020-2.0/PMBB-Release-2020-2.0_genetic_imputed-topmed-r2_chrAll_1kg_hm3_eur_newID_EUR \
--score /home/mlreed/PRSCalcs/EUR/Cardio/prscs_EUR_Nelson2017EUR_Cardio_pst_eff_a1_b0.5_phiauto_chrALL.txt 2 4 6 \
--out PRS_prscs_EUR_Cardio

## Lipids Phenotypes - EUR Only 

#####  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Switch to R kernel 

#### *HDL*

In [ ]:
library(data.table)
HDL_data <- fread('/project/rkemberlab/summary_stats/EUR/Willer_2013_HDL_EUR_GLGC.txt')
LDL_data <- fread('/project/rkemberlab/summary_stats/EUR/Willer_2013_LDL_EUR_GLGC.txt')
TgData <- fread('/project/rkemberlab/summary_stats/EUR/Willer_2013_TG_EUR_GLGC.txt')
TcData <- fread('/project/rkemberlab/summary_stats/EUR/Willer_2013_TC_EUR_GLGC.txt')

In [ ]:
head(HDL_data)

In [ ]:
HDLDf <- HDL_data[,c('rsid','A1','A2','beta','P-value')] #ouput file names
names(HDLDf) <- c('SNP','A1','A2','BETA','P')
# Upper the Allele
HDLDf$A1 <- toupper(HDLDf$A1)
HDLDf$A2 <- toupper(HDLDf$A2)
fwrite(HDLDf,'/project/rkemberlab/summary_stats/reformatted_sumstats/prscs_prscsx/hdl_eur_Willer2013_reformatted_sumstats.txt',row.names = F,col.names = T,sep = '\t')
head(HDLDf)

In [ ]:
N <- HDL_data$N
N <- as.numeric(N)
N <- na.omit(N)
max(N)

#####  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Switch to bash kernel 

In [ ]:
cat /project/rkemberlab/scripts/run_prscs_eur.sh

In [ ]:
sh /project/rkemberlab/scripts/run_prscs_eur.sh /project/rkemberlab/summary_stats/reformatted_sumstats/prscs_prscsx/hdl_eur_Willer2013_reformatted_sumstats.txt 187167 /home/mlreed/PRSCalcs/EUR/HDL/ Willer2013EUR_HDL  

In [ ]:
cat /home/mlreed/PRSCalcs/EUR/HDL/prscs_EUR_Willer2013EUR_HDL_pst_eff_a1_b0.5_phiauto_chr* > /home/mlreed/PRSCalcs/EUR/HDL/prscs_EUR_Willer2013EUR_HDL_pst_eff_a1_b0.5_phiauto_chrALL.txt

In [ ]:
module load plink-1.90b3b
plink \
--bfile /project/rkemberlab/datasets/PMBB/Genotype/PMBB-Release-2020-2.0/PMBB-Release-2020-2.0_genetic_imputed-topmed-r2_chrAll_1kg_hm3_eur_newID_EUR \
--score /home/mlreed/PRSCalcs/EUR/HDL/prscs_EUR_Willer2013EUR_HDL_pst_eff_a1_b0.5_phiauto_chrALL.txt 2 4 6 \
--out PRS_prscs_EUR_HDL

#### *LDL*

In [ ]:
library(data.table)
head(LDL_data)
LDLDf <- LDL_data[,c('rsid','A1','A2','beta','P-value')] #ouput file names
names(LDLDf) <- c('SNP','A1','A2','BETA','P')
# Upper the Allele
LDLDf$A1 <- toupper(LDLDf$A1)
LDLDf$A2 <- toupper(LDLDf$A2)
fwrite(LDLDf,'/project/rkemberlab/summary_stats/reformatted_sumstats/prscs_prscsx/ldl_eur_Willer2013_reformatted_sumstats.txt',row.names = F,col.names = T,sep = '\t')
head(LDLDf)

In [ ]:
N <- LDL_data$N
N <- as.numeric(N)
N <- na.omit(N)
max(N)

#####  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Switch to bash kernel 

In [ ]:
sh /project/rkemberlab/scripts/run_prscs_eur.sh /project/rkemberlab/summary_stats/reformatted_sumstats/prscs_prscsx/ldl_eur_Willer2013_reformatted_sumstats.txt 173082 /home/mlreed/PRSCalcs/EUR/LDL/ Willer2013EUR_LDL  

In [ ]:
cat /home/mlreed/PRSCalcs/EUR/LDL/prscs_EUR_Willer2013EUR_LDL_pst_eff_a1_b0.5_phiauto_chr* > /home/mlreed/PRSCalcs/EUR/LDL/prscs_EUR_Willer2013EUR_LDL_pst_eff_a1_b0.5_phiauto_chrALL.txt

In [ ]:
module load plink-1.90b3b
plink \
--bfile /project/rkemberlab/datasets/PMBB/Genotype/PMBB-Release-2020-2.0/PMBB-Release-2020-2.0_genetic_imputed-topmed-r2_chrAll_1kg_hm3_eur_newID_EUR \
--score /home/mlreed/PRSCalcs/EUR/LDL/prscs_EUR_Willer2013EUR_LDL_pst_eff_a1_b0.5_phiauto_chrALL.txt 2 4 6 \
--out PRS_prscs_EUR_LDL

#### *Total Tryglicerides*

In [ ]:
tgDf <- TgData[,c('rsid','A1','A2','beta','P-value')] #ouput file names
names(tgDf) <- c('SNP','A1','A2','BETA','P')
# Upper the Allele
tgDf$A1 <- toupper(tgDf$A1)
tgDf$A2 <- toupper(tgDf$A2)
fwrite(tgDf,'/project/rkemberlab/summary_stats/reformatted_sumstats/prscs_prscsx/tg_eur_Willer2013_reformatted_sumstats.txt',row.names = F,col.names = T,sep = '\t')
head(TgData)
head(tgDf)

In [ ]:
N <- TgData$N
N <- as.numeric(N)
N <- na.omit(N)
max(N)

#####  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Switch to bash kernel 

In [ ]:
sh /project/rkemberlab/scripts/run_prscs_eur.sh /project/rkemberlab/summary_stats/reformatted_sumstats/prscs_prscsx/tg_eur_Willer2013_reformatted_sumstats.txt 177861 /home/mlreed/PRSCalcs/EUR/TriGlTotal/ Willer2013EUR_Tg  

In [ ]:
cat /home/mlreed/PRSCalcs/EUR/TriGlTotal/prscs_EUR_Willer2013EUR_Tg_pst_eff_a1_b0.5_phiauto_chr* > /home/mlreed/PRSCalcs/EUR/TriGlTotal/prscs_EUR_Willer2013EUR_Tg_pst_eff_a1_b0.5_phiauto_chrALL.txt

In [ ]:
module load plink-1.90b3b
plink \
--bfile /project/rkemberlab/datasets/PMBB/Genotype/PMBB-Release-2020-2.0/PMBB-Release-2020-2.0_genetic_imputed-topmed-r2_chrAll_1kg_hm3_eur_newID_EUR \
--score /home/mlreed/PRSCalcs/EUR/TriGlTotal/prscs_EUR_Willer2013EUR_Tg_pst_eff_a1_b0.5_phiauto_chrALL.txt 2 4 6 \
--out PRS_prscs_EUR_tg

#### *Total Cholesterol*

In [ ]:
tcDf <- TcData[,c('rsid','A1','A2','beta','P-value')] #ouput file names
names(tcDf) <- c('SNP','A1','A2','BETA','P')
# Upper the Allele
tcDf$A1 <- toupper(tcDf$A1)
tcDf$A2 <- toupper(tcDf$A2)
fwrite(tcDf,'/project/rkemberlab/summary_stats/reformatted_sumstats/prscs_prscsx/tc_eur_Willer2013_reformatted_sumstats.txt',row.names = F,col.names = T,sep = '\t')
head(TcData)
head(tcDf)

In [ ]:
N <- TcData$N
N <- as.numeric(N)
N <- na.omit(N)
max(N)

#####  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Switch to bash kernel 

In [ ]:
sh /project/rkemberlab/scripts/run_prscs_eur.sh /project/rkemberlab/summary_stats/reformatted_sumstats/prscs_prscsx/tc_eur_Willer2013_reformatted_sumstats.txt 187365 /home/mlreed/PRSCalcs/EUR/CholTot/ Willer2013EUR_Tc  

In [ ]:
cat /home/mlreed/PRSCalcs/EUR/CholTot/prscs_EUR_Willer2013EUR_Tc_pst_eff_a1_b0.5_phiauto_chr* > /home/mlreed/PRSCalcs/EUR/CholTot/prscs_EUR_Willer2013EUR_Tc_pst_eff_a1_b0.5_phiauto_chrALL.txt

In [ ]:
module load plink-1.90b3b
plink \
--bfile /project/rkemberlab/datasets/PMBB/Genotype/PMBB-Release-2020-2.0/PMBB-Release-2020-2.0_genetic_imputed-topmed-r2_chrAll_1kg_hm3_eur_newID_EUR \
--score /home/mlreed/PRSCalcs/EUR/CholTot/prscs_EUR_Willer2013EUR_Tc_pst_eff_a1_b0.5_phiauto_chrALL.txt 2 4 6 \
--out PRS_prscs_EUR_tc

# Linear Regression

#####  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Switch to R kernel 

####  T2D EUR

In [ ]:
library(data.table)
library(dplyr)

In [ ]:
pheno <- read.table("/project/rkemberlab/datasets/PMBB/PMBB_Diagnosis_Deidentified_072020_Phecodes.txt", header = TRUE)
head(pheno)
T2DPhecodeDf <- pheno[, c("id", "X250.2")]
head(T2DPhecodeDf)
colnames(T2DPhecodeDf) = c("PMBB_ID", "pheno")
head(T2DPhecodeDf)

In [ ]:
cov <- read.table("/project/rkemberlab/datasets/PMBB/PMBB_eur_covariates.txt", header = TRUE, sep = '\t')
head(cov)
cov_age_pcs<- merge(T2DPhecodeDf, cov, by = "PMBB_ID")
head(cov_age_pcs)

In [ ]:
T2DEUR_prs <- read.table("/home/mlreed/PRSCalcs/EUR/T2D/PRS_prscs_EUR_T2D.profile", header = TRUE)
head(T2DEUR_prs)
T2DEUR_prs <- T2DEUR_prs[,c(2,6)]
colnames(T2DEUR_prs) <- c("PMBB_ID", "PRS")
T2DEUR_prs <- T2DEUR_prs %>% mutate(PRS_scaled = scale(PRS))
head(T2DEUR_prs)
T2D_pheno_cov_prs <- merge(cov_age_pcs, T2DEUR_prs, by = "PMBB_ID")
head(T2D_pheno_cov_prs)

In [ ]:
T2DMOD_EUR <- glm(formula = as.factor(T2D_pheno_cov_prs$pheno) ~ T2D_pheno_cov_prs$PRS_scaled + 
    T2D_pheno_cov_prs$age + T2D_pheno_cov_prs$Gen_Sex + T2D_pheno_cov_prs$EUR_PC1 + 
    T2D_pheno_cov_prs$EUR_PC2 + T2D_pheno_cov_prs$EUR_PC3 + T2D_pheno_cov_prs$EUR_PC4 + 
    T2D_pheno_cov_prs$EUR_PC5 + T2D_pheno_cov_prs$EUR_PC6 + T2D_pheno_cov_prs$EUR_PC7 + 
    T2D_pheno_cov_prs$EUR_PC8 + T2D_pheno_cov_prs$EUR_PC9 + T2D_pheno_cov_prs$EUR_PC10, 
    family = "binomial")

In [ ]:
#view results summary
summary(T2DMOD_EUR)

#covert beta value ('Estimate') into odds ratio since our outcome is binary
paste("OR:",exp(summary(T2DMOD_EUR)$coefficients[2]))

#create upper and lower beta confidence intervals
paste("Lower CI:",exp((summary(T2DMOD_EUR)$coefficients[2])-1.96*summary(T2DMOD_EUR)$coefficients[2,2]))
paste("Upper CI:" ,exp((summary(T2DMOD_EUR)$coefficients[2])+1.96*summary(T2DMOD_EUR)$coefficients[2,2]))

#generate p value
paste("P-value:" ,summary(T2DMOD_EUR)$coefficients[2,4])

#### T2D AFR

In [ ]:
covafr <- read.table("/project/rkemberlab/datasets/PMBB/PMBB_afr_covariates.txt", header = TRUE, sep = '\t')
head(covafr)
covafr_age_pcs<- merge(T2DPhecodeDf, covafr, by = "PMBB_ID")
head(covafr_age_pcs)

In [ ]:
T2DAFR_prs <- read.table("/home/mlreed/PRSCalcs/AFR/T2D/PRS_prscs_AFR_T2D.profile", header = TRUE)
head(T2DAFR_prs)
T2DAFR_prs <- T2DAFR_prs[,c(2,6)]
colnames(T2DAFR_prs) <- c("PMBB_ID", "PRS")
T2DAFR_prs <- T2DAFR_prs %>% mutate(PRS_scaled = scale(PRS))
head(T2DAFR_prs)
T2D_pheno_covafr_prs <- merge(covafr_age_pcs, T2DAFR_prs, by = "PMBB_ID")
head(T2D_pheno_covafr_prs)

In [ ]:
T2DMOD_AFR <- glm(formula = as.factor(T2D_pheno_covafr_prs$pheno) ~ T2D_pheno_covafr_prs$PRS_scaled + 
    T2D_pheno_covafr_prs$age + T2D_pheno_covafr_prs$Gen_Sex + T2D_pheno_covafr_prs$AFR_PC1 + 
    T2D_pheno_covafr_prs$AFR_PC2 + T2D_pheno_covafr_prs$AFR_PC3 + T2D_pheno_covafr_prs$AFR_PC4 + 
    T2D_pheno_covafr_prs$AFR_PC5 + T2D_pheno_covafr_prs$AFR_PC6 + T2D_pheno_covafr_prs$AFR_PC7 + 
    T2D_pheno_covafr_prs$AFR_PC8 + T2D_pheno_covafr_prs$AFR_PC9 + T2D_pheno_covafr_prs$AFR_PC10, 
    family = "binomial")

In [ ]:
#view results summary
summary(T2DMOD_AFR)

#covert beta value ('Estimate') into odds ratio since our outcome is binary
paste("OR:",exp(summary(T2DMOD_AFR)$coefficients[2]))

#create upper and lower beta confidence intervals
paste("Lower CI:",exp((summary(T2DMOD_AFR)$coefficients[2])-1.96*summary(T2DMOD_AFR)$coefficients[2,2]))
paste("Upper CI:" ,exp((summary(T2DMOD_AFR)$coefficients[2])+1.96*summary(T2DMOD_AFR)$coefficients[2,2]))

#generate p value
paste("P-value:" ,summary(T2DMOD_AFR)$coefficients[2,4])

####  HDL EUR

In [ ]:
pheno <- read.table("/project/rkemberlab/datasets/PMBB/PMBB_Diagnosis_Deidentified_072020_Phecodes.txt", header = TRUE)
HDLPhecodeDf <- pheno[, c("id", "X272.1")]
colnames(HDLPhecodeDf) = c("PMBB_ID", "pheno")
head(HDLPhecodeDf)

In [ ]:
cov <- read.table("/project/rkemberlab/datasets/PMBB/PMBB_eur_covariates.txt", header = TRUE, sep = '\t')
head(cov)
cov_age_pcs<- merge(HDLPhecodeDf, cov, by = "PMBB_ID")
head(cov_age_pcs)

In [ ]:
HDLEUR_prs <- read.table("/home/mlreed/PRSCalcs/EUR/HDL/PRS_prscs_EUR_HDL.profile", header = TRUE)
head(HDLEUR_prs)
HDLEUR_prs <- HDLEUR_prs[,c(2,6)]
colnames(HDLEUR_prs) <- c("PMBB_ID", "PRS")
HDLEUR_prs <- HDLEUR_prs %>% mutate(PRS_scaled = scale(PRS))
head(HDLEUR_prs)
HDL_pheno_cov_prs <- merge(cov_age_pcs, HDLEUR_prs, by = "PMBB_ID")
head(HDL_pheno_cov_prs)

In [ ]:
HDLMOD_EUR <- glm(formula = as.factor(HDL_pheno_cov_prs$pheno) ~ HDL_pheno_cov_prs$PRS_scaled + 
    HDL_pheno_cov_prs$age + HDL_pheno_cov_prs$Gen_Sex + HDL_pheno_cov_prs$EUR_PC1 + 
    HDL_pheno_cov_prs$EUR_PC2 + HDL_pheno_cov_prs$EUR_PC3 + HDL_pheno_cov_prs$EUR_PC4 + 
    HDL_pheno_cov_prs$EUR_PC5 + HDL_pheno_cov_prs$EUR_PC6 + HDL_pheno_cov_prs$EUR_PC7 + 
   HDL_pheno_cov_prs$EUR_PC8 + HDL_pheno_cov_prs$EUR_PC9 + HDL_pheno_cov_prs$EUR_PC10, 
    family = "binomial")

In [ ]:
#view results summary
summary(HDLMOD_EUR)

#covert beta value ('Estimate') into odds ratio since our outcome is binary
paste("OR:",exp(summary(HDLMOD_EUR)$coefficients[2]))

#create upper and lower beta confidence intervals
paste("Lower CI:",exp((summary(HDLMOD_EUR)$coefficients[2])-1.96*summary(HDLMOD_EUR)$coefficients[2,2]))
paste("Upper CI:" ,exp((summary(HDLMOD_EUR)$coefficients[2])+1.96*summary(HDLMOD_EUR)$coefficients[2,2]))

#generate p value
paste("P-value:" ,summary(HDLMOD_EUR)$coefficients[2,4])

#### LDL EUR

In [ ]:
#pheno <- read.table("/project/rkemberlab/datasets/PMBB/PMBB_Diagnosis_Deidentified_072020_Phecodes.txt", header = TRUE)
LDLPhecodeDf <- pheno[, c("id", "X272.11")]
colnames(LDLPhecodeDf) = c("PMBB_ID", "pheno")
head(LDLPhecodeDf)

In [ ]:
cov <- read.table("/project/rkemberlab/datasets/PMBB/PMBB_eur_covariates.txt", header = TRUE, sep = '\t')
head(cov)
cov_age_pcs<- merge(LDLPhecodeDf, cov, by = "PMBB_ID")
head(cov_age_pcs)

In [ ]:
LDLEUR_prs <- read.table("/home/mlreed/PRSCalcs/EUR/LDL/PRS_prscs_EUR_LDL.profile", header = TRUE)
head(LDLEUR_prs)
LDLEUR_prs <- LDLEUR_prs[,c(2,6)]
colnames(LDLEUR_prs) <- c("PMBB_ID", "PRS")
LDLEUR_prs <- LDLEUR_prs %>% mutate(PRS_scaled = scale(PRS))
head(LDLEUR_prs)
LDL_pheno_cov_prs <- merge(cov_age_pcs, LDLEUR_prs, by = "PMBB_ID")
head(LDL_pheno_cov_prs)

In [ ]:
LDLMOD_EUR <- glm(formula = as.factor(LDL_pheno_cov_prs$pheno) ~ LDL_pheno_cov_prs$PRS_scaled + 
    LDL_pheno_cov_prs$age + LDL_pheno_cov_prs$Gen_Sex + HDL_pheno_cov_prs$EUR_PC1 + 
    LDL_pheno_cov_prs$EUR_PC2 + LDL_pheno_cov_prs$EUR_PC3 + LDL_pheno_cov_prs$EUR_PC4 + 
    LDL_pheno_cov_prs$EUR_PC5 + LDL_pheno_cov_prs$EUR_PC6 + LDL_pheno_cov_prs$EUR_PC7 + 
    LDL_pheno_cov_prs$EUR_PC8 + LDL_pheno_cov_prs$EUR_PC9 + LDL_pheno_cov_prs$EUR_PC10, 
    family = "binomial")

In [ ]:
#view results summary
summary(LDLMOD_EUR)

#covert beta value ('Estimate') into odds ratio since our outcome is binary
paste("OR:",exp(summary(LDLMOD_EUR)$coefficients[2]))

#create upper and lower beta confidence intervals
paste("Lower CI:",exp((summary(LDLMOD_EUR)$coefficients[2])-1.96*summary(LDLMOD_EUR)$coefficients[2,2]))
paste("Upper CI:" ,exp((summary(LDLMOD_EUR)$coefficients[2])+1.96*summary(LDLMOD_EUR)$coefficients[2,2]))

#generate p value
paste("P-value:" ,summary(LDLMOD_EUR)$coefficients[2,4])

#### Tc EUR

In [ ]:
#pheno <- read.table("/project/rkemberlab/datasets/PMBB/PMBB_Diagnosis_Deidentified_072020_Phecodes.txt", header = TRUE)
TcPhecodeDf <- pheno[, c("id", "X272.1")]
colnames(TcPhecodeDf) = c("PMBB_ID", "pheno")
head(TcPhecodeDf)

cov <- read.table("/project/rkemberlab/datasets/PMBB/PMBB_eur_covariates.txt", header = TRUE, sep = '\t')
head(cov)
cov_age_pcs<- merge(TcPhecodeDf, cov, by = "PMBB_ID")
head(cov_age_pcs)

TcEUR_prs <- read.table("/home/mlreed/PRSCalcs/EUR/CholTot/PRS_prscs_EUR_tc.profile", header = TRUE)
head(TcEUR_prs)
TcEUR_prs <- TcEUR_prs[,c(2,6)]
colnames(TcEUR_prs) <- c("PMBB_ID", "PRS")
TcEUR_prs <- TcEUR_prs %>% mutate(PRS_scaled = scale(PRS))
head(TcEUR_prs)
Tc_pheno_cov_prs <- merge(cov_age_pcs, TcEUR_prs, by = "PMBB_ID")
head(Tc_pheno_cov_prs)


In [ ]:
TcMOD_EUR <- glm(formula = as.factor(Tc_pheno_cov_prs$pheno) ~ Tc_pheno_cov_prs$PRS_scaled + 
    Tc_pheno_cov_prs$age + Tc_pheno_cov_prs$Gen_Sex + HDL_pheno_cov_prs$EUR_PC1 + 
    Tc_pheno_cov_prs$EUR_PC2 + Tc_pheno_cov_prs$EUR_PC3 + Tc_pheno_cov_prs$EUR_PC4 + 
    Tc_pheno_cov_prs$EUR_PC5 + Tc_pheno_cov_prs$EUR_PC6 + Tc_pheno_cov_prs$EUR_PC7 + 
    Tc_pheno_cov_prs$EUR_PC8 + Tc_pheno_cov_prs$EUR_PC9 + Tc_pheno_cov_prs$EUR_PC10, 
    family = "binomial")

In [ ]:
#view results summary
summary(TcMOD_EUR)

#covert beta value ('Estimate') into odds ratio since our outcome is binary
paste("OR:",exp(summary(TcMOD_EUR)$coefficients[2]))

#create upper and lower beta confidence intervals
paste("Lower CI:",exp((summary(TcMOD_EUR)$coefficients[2])-1.96*summary(TcMOD_EUR)$coefficients[2,2]))
paste("Upper CI:" ,exp((summary(TcMOD_EUR)$coefficients[2])+1.96*summary(TcMOD_EUR)$coefficients[2,2]))

#generate p value
paste("P-value:" ,summary(TcMOD_EUR)$coefficients[2,4])

#### Tg EUR

In [ ]:
#pheno <- read.table("/project/rkemberlab/datasets/PMBB/PMBB_Diagnosis_Deidentified_072020_Phecodes.txt", header = TRUE)
TgPhecodeDf <- pheno[, c("id", "X270.1")]
colnames(TgPhecodeDf) = c("PMBB_ID", "pheno")
head(TgPhecodeDf)

cov <- read.table("/project/rkemberlab/datasets/PMBB/PMBB_eur_covariates.txt", header = TRUE, sep = '\t')
head(cov)
cov_age_pcs<- merge(TgPhecodeDf, cov, by = "PMBB_ID")
head(cov_age_pcs)

TgEUR_prs <- read.table("/home/mlreed/PRSCalcs/EUR/TriGlTotal/PRS_prscs_EUR_tg.profile", header = TRUE)
head(TgEUR_prs)
TgEUR_prs <- TgEUR_prs[,c(2,6)]
colnames(TgEUR_prs) <- c("PMBB_ID", "PRS")
TgEUR_prs <- TgEUR_prs %>% mutate(PRS_scaled = scale(PRS))
head(TgEUR_prs)
Tg_pheno_cov_prs <- merge(cov_age_pcs, TgEUR_prs, by = "PMBB_ID")
head(Tg_pheno_cov_prs)

In [ ]:
TgMOD_EUR <- glm(formula = as.factor(Tg_pheno_cov_prs$pheno) ~ Tg_pheno_cov_prs$PRS_scaled + 
    Tg_pheno_cov_prs$age + Tg_pheno_cov_prs$Gen_Sex + HDL_pheno_cov_prs$EUR_PC1 + 
    Tg_pheno_cov_prs$EUR_PC2 + Tg_pheno_cov_prs$EUR_PC3 + Tg_pheno_cov_prs$EUR_PC4 + 
    Tg_pheno_cov_prs$EUR_PC5 + Tg_pheno_cov_prs$EUR_PC6 + Tg_pheno_cov_prs$EUR_PC7 + 
    Tg_pheno_cov_prs$EUR_PC8 + Tg_pheno_cov_prs$EUR_PC9 + Tg_pheno_cov_prs$EUR_PC10, 
    family = "binomial")

In [ ]:
#view results summary
summary(TgMOD_EUR)

#covert beta value ('Estimate') into odds ratio since our outcome is binary
paste("OR:",exp(summary(TgMOD_EUR)$coefficients[2]))

#create upper and lower beta confidence intervals
paste("Lower CI:",exp((summary(TgMOD_EUR)$coefficients[2])-1.96*summary(TgMOD_EUR)$coefficients[2,2]))
paste("Upper CI:" ,exp((summary(TgMOD_EUR)$coefficients[2])+1.96*summary(TgMOD_EUR)$coefficients[2,2]))

#generate p value
paste("P-value:" ,summary(TgMOD_EUR)$coefficients[2,4])

#### Cardio EUR

In [ ]:
#pheno <- read.table("/project/rkemberlab/datasets/PMBB/PMBB_Diagnosis_Deidentified_072020_Phecodes.txt", header = TRUE)
CardioPhecodeDf <- pheno[, c("id", "X411.4")]
colnames(CardioPhecodeDf) = c("PMBB_ID", "pheno")
head(CardioPhecodeDf)

cov <- read.table("/project/rkemberlab/datasets/PMBB/PMBB_eur_covariates.txt", header = TRUE, sep = '\t')
head(cov)
cov_age_pcs<- merge(CardioPhecodeDf, cov, by = "PMBB_ID")
head(cov_age_pcs)

CardioEUR_prs <- read.table("/home/mlreed/PRSCalcs/EUR/Cardio/PRS_prscs_EUR_Cardio.profile", header = TRUE)
head(CardioEUR_prs)
CardioEUR_prs <- CardioEUR_prs[,c(2,6)]
colnames(CardioEUR_prs) <- c("PMBB_ID", "PRS")
CardioEUR_prs <- CardioEUR_prs %>% mutate(PRS_scaled = scale(PRS))
head(CardioEUR_prs)
Cardio_pheno_cov_prs <- merge(cov_age_pcs, CardioEUR_prs, by = "PMBB_ID")
head(Cardio_pheno_cov_prs)

In [ ]:
CardioMOD_EUR <- glm(formula = as.factor(Cardio_pheno_cov_prs$pheno) ~ Cardio_pheno_cov_prs$PRS_scaled + 
    Cardio_pheno_cov_prs$age + Cardio_pheno_cov_prs$Gen_Sex + HDL_pheno_cov_prs$EUR_PC1 + 
    Cardio_pheno_cov_prs$EUR_PC2 + Cardio_pheno_cov_prs$EUR_PC3 + Cardio_pheno_cov_prs$EUR_PC4 + 
    Cardio_pheno_cov_prs$EUR_PC5 + Cardio_pheno_cov_prs$EUR_PC6 + Cardio_pheno_cov_prs$EUR_PC7 + 
    Cardio_pheno_cov_prs$EUR_PC8 + Cardio_pheno_cov_prs$EUR_PC9 + Cardio_pheno_cov_prs$EUR_PC10, 
    family = "binomial")

In [ ]:
#view results summary
summary(CardioMOD_EUR)

#covert beta value ('Estimate') into odds ratio since our outcome is binary
paste("OR:",exp(summary(CardioMOD_EUR)$coefficients[2]))

#create upper and lower beta confidence intervals
paste("Lower CI:",exp((summary(CardioMOD_EUR)$coefficients[2])-1.96*summary(CardioMOD_EUR)$coefficients[2,2]))
paste("Upper CI:" ,exp((summary(CardioMOD_EUR)$coefficients[2])+1.96*summary(CardioMOD_EUR)$coefficients[2,2]))

#generate p value
paste("P-value:" ,summary(CardioMOD_EUR)$coefficients[2,4])